## 1. Importing the datasets and libraries
---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_yelp = pd.read_csv('yelp_labelled.txt', sep = '\t', header = None)

In [3]:
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
# Assign Column names
column_names = ['Review', 'Sentiment']
data_yelp.columns = column_names

In [5]:
data_yelp.head()
# it seems much better

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [6]:
data_amazon = pd.read_csv('amazon_cells_labelled.txt', sep = '\t', header = None)
data_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [8]:
data_amazon.columns = column_names
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [9]:
data_imdb = pd.read_csv('imdb_labelled.txt', sep = '\t', header = None)
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [11]:
data_imdb.columns = column_names
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [12]:
# Append all the data in a single dataframe
data = data_yelp.append([data_amazon, data_imdb], ignore_index= True)
data.shape

(2748, 2)

In [13]:
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [14]:
# Check distribution of sentiments
data['Sentiment'].value_counts()
# 1386 positive
# 1362 negative

1    1386
0    1362
Name: Sentiment, dtype: int64

In [16]:
# check for null values
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [17]:
x = data['Review']
y = data['Sentiment']

## 2. Data Cleaning
---

In [ ]:
# Here I will remove stopwords, punctuations 
# as well as I will apply lemmatization

#### => Create a function to clean the data

In [18]:
import string
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)

In [23]:
import spacy
nlp = spacy.load('en_core_web_sm')
# creating a function with them 
def text_cleaning(sentence):
  doc = nlp(sentence)
  #list of tokens
  tokens = []
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens
# if root form of that word is not pronoun then it is foing to convert that into lower form
# and if that word is a proper noun, then we are directly taking lower form, because there is no lemma

In [25]:
text_cleaning("Hello all, It's a beautiful day outside there!")

['hello', 'beautiful', 'day', 'outside']

#### => Vectorization Feature Engineering (TF-IDF)

In [29]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [31]:
tfidf = TfidfVectorizer(tokenizer = text_cleaning)

# 3. Train the model

In [32]:
classifier = LinearSVC()

#### => Splitting the dataset into the Train and Test set

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(x,y, test_size = 0.2, random_state =123)

In [34]:
X_train.shape, X_test.shape

((2198,), (550,))

In [35]:
# fit the trains
clf = Pipeline([('tfidf', tfidf), ('classifier', classifier)])
# it will first do vectorization and then it will do classification

In [36]:
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_cleaning at 0x7f2fa4df1488>,
                                 use_idf=True, vocabulary=None)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_interce

# 4. Prediction
---

In [37]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [38]:
y_pred = clf.predict(X_test)

In [39]:
# conf matrix
confusion_matrix(y_test, y_pred)

array([[215,  62],
       [ 45, 228]])

In [40]:
# accuracy score
accuracy_score(y_test, y_pred)

0.8054545454545454

In [41]:
# Lets predict soome sample sentences
clf.predict(["Now, I am learning NLP in fun fashion!"])
# it could be 1, it means review is posiive

array([1])

In [42]:
clf.predict(["It's hard to learn new things!"])
# output is 0, it means it is negative

array([0])